# Bibliotecas

In [39]:
import ranking

import pandas as pd
pd.set_option('display.max_colwidth', None)

import numpy as np
from math import ceil

from sklearn.model_selection import train_test_split

# Constantes

In [2]:
COLUNAS = ("descricao_1", "ean_1", "titulo_1", "url_1",	"titulo_cb_1", "loja_1",
           "descricao_2", "ean_2", "titulo_2", "url_2",	"titulo_cb_2", "loja_2",
           "match")
#NUM_AMOSTRAS = 280
DIRETORIO = "Dados/Datasets/"

#lista_nomes = ["celulares", "notebooks", "geladeiras", "fogoes", "tvs"]
lista_nomes = ["fogoes"]

In [3]:
df = pd.read_csv(f"Dados/Produtos Processados/pp_{lista_nomes[0]}.csv")
df['ean'].isna().sum()

0

In [4]:
produto = lista_nomes[0]

# Montando os Pares e os Não Pares

Datasets que vou montar:

    1. Hard negative dataset
        Não matches são feitos de acordo com o algoritmo de rankeamento. Pega o não-match mais próximo
    2. Aleatório
        Não matches são feitos aleatóriamente dentro da categoria

In [5]:
def criar_dicionario(descricao_1, ean_1, titulo_1, url_1, titulo_cb_1, loja_1,
                     descricao_2, ean_2, titulo_2, url_2, titulo_cb_2, loja_2,
                     match):
    
    return {'descricao_1' : descricao_1 , 'ean_1' : ean_1, 'titulo_1' : titulo_1,
            'url_1' : url_1, 'titulo_cb_1' : titulo_cb_1, 'loja_1' : loja_1,
            
            'descricao_2' : descricao_2 , 'ean_2' : ean_2, 'titulo_2' : titulo_2,
            'url_2' : url_2 , 'titulo_cb_2' : titulo_cb_2, 'loja_2' : loja_2,
            
            'match' : match
           }

## Pares

In [6]:
def criar_pares(colunas, df_final_dup):

    df_pares = pd.DataFrame(columns = colunas)
    df_pares

    lista_ean = []

    for i in range(len(df_final_dup)):
        
        if df_final_dup.iloc[i]['ean'] not in lista_ean:
            
            lista_ean.append(df_final_dup.iloc[i]['ean'])
            
            indice = df_final_dup.index[df_final_dup['ean'] == df_final_dup.iloc[i]['ean']].tolist()
            
            for j in range(len(indice)-1):
                
                for k in range(j + 1, len(indice)):
                    
                    dicionario = criar_dicionario(descricao_1  = df_final_dup.loc[indice[j]]['descricao'] , ean_1 = df_final_dup.loc[indice[j]]['ean'],
                                                  titulo_1 = df_final_dup.loc[indice[j]]['titulo'], url_1 = df_final_dup.loc[indice[j]]['url'],
                                                  titulo_cb_1 = df_final_dup.loc[indice[j]]['titulo_cb'], loja_1 = df_final_dup.loc[indice[j]]['loja'],
                                                  
                                                  descricao_2 = df_final_dup.loc[indice[k]]['descricao'] , ean_2 = df_final_dup.loc[indice[k]]['ean'],
                                                  titulo_2 = df_final_dup.loc[indice[k]]['titulo'], url_2 = df_final_dup.loc[indice[k]]['url'],
                                                  titulo_cb_2 = df_final_dup.loc[indice[k]]['titulo_cb'], loja_2 = df_final_dup.loc[indice[k]]['loja'],
                                                  
                                                  match = 1
                                                  )
                
                    df_pares = df_pares.append(dicionario, ignore_index = True)

    return df_pares

In [7]:
#df_pares = criar_pares(COLUNAS, df)
#df_pares[df_pares['ean_1'] != df_pares['ean_2']]

In [8]:
#df_pares.to_csv(f"Dados/Pares/pares_{produto}.csv", index = False)

## Não Pares

In [9]:
df_pares = pd.read_csv(f"Dados/Pares/pares_{produto}.csv")

### Aleatório

In [10]:
def criar_nao_pares(df_pares, qtd, colunas = COLUNAS):

    n_matches = df_pares.shape[0]*qtd
    df_nao_pares_aleatorio = pd.DataFrame(columns = colunas)

    for i in range(n_matches):
        created_match = False
        
        while(not created_match):
            df_sorteado = df.sample(n = 2)
            if(df_sorteado.iloc[0]['ean'] != df_sorteado.iloc[1]['ean']):
                
                dic = criar_dicionario(
                                       df_sorteado.iloc[0]['descricao'], df_sorteado.iloc[0]['ean'], df_sorteado.iloc[0]['titulo'], df_sorteado.iloc[0]['url'], df_sorteado.iloc[0]['titulo_cb'], df_sorteado.iloc[0]['loja'],
                                       df_sorteado.iloc[1]['descricao'], df_sorteado.iloc[1]['ean'], df_sorteado.iloc[1]['titulo'], df_sorteado.iloc[1]['url'], df_sorteado.iloc[1]['titulo_cb'], df_sorteado.iloc[1]['loja'],
                                       0
                                      )
                
                df_nao_pares_aleatorio = df_nao_pares_aleatorio.append(dic, ignore_index = True)
                created_match = True

    return df_nao_pares_aleatorio

#### 5 : 1

In [11]:
#df_nao_pares_aleatorio_5 = criar_nao_pares(df_pares, 5)

In [12]:
#df_nao_pares_aleatorio_5.to_csv(f"Dados/Não Pares/Aléatorio/5 por 1/nao_pares_{produto}.csv", index = False)

#### 1 : 1

In [13]:
#df_nao_pares_aleatorio_1 = criar_nao_pares(df_pares, 1)

In [14]:
#df_nao_pares_aleatorio_1.to_csv(f"Dados/Não Pares/Aléatorio/1 por 1/nao_pares_{produto}.csv", index = False)

#### Conferindo

In [15]:
'''produto = 'celulares'
p = pd.read_csv(f"Dados/Pares/pares_{produto}.csv")
n = pd.read_csv(f"Dados/Não Pares/Aléatorio/1 por 1/nao_pares_{produto}.csv")

print(f"{p.size}\n{n.size}")'''

'produto = \'celulares\'\np = pd.read_csv(f"Dados/Pares/pares_{produto}.csv")\nn = pd.read_csv(f"Dados/Não Pares/Aléatorio/1 por 1/nao_pares_{produto}.csv")\n\nprint(f"{p.size}\n{n.size}")'

## Hard Negative

### Funções

In [16]:
def criar_lista_nao_pares_hn(df, n_matches, ean_repetido, indices_resultado, colunas = COLUNAS):

    lista_final = []
    df_nao_pares_hn = pd.DataFrame(columns = colunas)

    limite = ceil(n_matches/len(ean_repetido))

    for i in range(len(ean_repetido)):

        lista_indice_top = []

        ean = ean_repetido[i]

        for indice_df in df[df['ean'] == ean].index.values: # todos os indices do dataframe onde tem o EAN repetido

            lista_top_nao_rank = []

            flag_encontrou = False
            j = 0
            cont = 0
            while (flag_encontrou == False):
                
                indice_rank = indices_resultado[indice_df][j]

                if df['ean'].loc[indice_df] != df['ean'].loc[indice_rank]:


                    lista_top_nao_rank.append(indice_rank)

                    cont += 1

                    if cont > limite:
                        flag_encontrou = True

                j += 1

            lista_indice_top.append([indice_df, lista_top_nao_rank])

        lista_final.append([ean, lista_indice_top])

    return lista_final

In [17]:
def criar_df_nao_pares_hn(df, n_matches, lista_nao_pares, colunas = COLUNAS):

    df_nao_pares = pd.DataFrame(columns = colunas)
    tam = len(np.array(lista_nao_pares)[:, 0])

    contador = 0
    voltas = 0
    while contador < n_matches:

        for i in range(tam):

            if len(lista_nao_pares[i][1]) > (voltas):
                    j = voltas
            else:
                    j = 0

            indice_1 = lista_nao_pares[i][1][j][0]
            indice_2 = lista_nao_pares[i][1][j][1][0]

            dicionario = criar_dicionario(
                                            descricao_1  = df.loc[indice_1]['descricao'] , ean_1 = df.loc[indice_1]['ean'],
                                            titulo_1 = df.loc[indice_1]['titulo'], url_1 = df.loc[indice_1]['url'],
                                            titulo_cb_1 = df.loc[indice_1]['titulo_cb'], loja_1 = df.loc[indice_1]['loja'],

                                            descricao_2 = df.loc[indice_2]['descricao'] , ean_2 = df.loc[indice_2]['ean'],
                                            titulo_2 = df.loc[indice_2]['titulo'], url_2 = df.loc[indice_2]['url'],
                                            titulo_cb_2 = df.loc[indice_2]['titulo_cb'], loja_2 = df.loc[indice_2]['loja'],

                                            match = 0
                                         )

            df_nao_pares = df_nao_pares.append(dicionario, ignore_index = True)

            del lista_nao_pares[i][1][j][1][0]

            contador += 1

            if contador == n_matches:
                break

        voltas += 1
        #print(voltas)

    return df_nao_pares


### Código

In [18]:
#df_pares = pd.read_csv(f"Dados/Pares/pares_{produto}.csv")

Descobrindo quais EAN se repetem

In [19]:
#vc = df['ean'].value_counts()
#ean_repetido = vc[vc > 1].index.values

In [20]:
# removendo a pontuação do título
#df["titulo_pp"] = df["titulo_cb"].apply(lambda x: ranking.remove_pontuacao(x))

# calculando o tamanho máximo do título
#tam_max = max(df.apply(lambda row: len(row["titulo_pp"]), axis = 1))

# calculando o BoW do título
#cv, titulo_bow = ranking.formatar_entrada_bow(df['titulo_pp'], mf = tam_max)   # np.unique(titulo_bow)

In [21]:
# calculando a distância entre os vetores
#resultado = ranking.calcular_dis_2_vetores(titulo_bow)

In [22]:
# ordenando o resultado
#indices, valores = ranking.ordenar_resultado(resultado)

In [23]:
#n_matches = df_pares.shape[0]

In [24]:
#lista_nao_pares = criar_lista_nao_pares_hn(df = df, n_matches = n_matches, ean_repetido = ean_repetido, indices_resultado = indices)

In [25]:
#df_nao_pares = criar_df_nao_pares_hn(df = df, n_matches = n_matches, lista_nao_pares = lista_nao_pares)
#print(f"Tamanho Pares:\t\t{df_pares.shape[0]}\nTamanho Não Pares:\t{df_nao_pares.shape[0]}")

In [26]:
#df_nao_pares.to_csv(f"Dados/Não Pares/Rankeado/nao_pares_{produto}.csv", index = False)

# Montando os Datasets

Datasets que vou montar:

    1. Hard negative dataset
        Não matches são feitos de acordo com o algoritmo de rankeamento. Pega o não-match mais próximo
    2. Aleatório
        Não matches são feitos aleatóriamente dentro da categoria

In [27]:
def salvar_arquivo(df, nome):
    df.to_csv(f"{DIRETORIO}{nome}.csv", index = False)


def embaralhar(df):
    
    df = df.sample(frac = 1)
    
    return df


def criar_dataset(lista_df, nome):
    
    df = pd.concat(lista_df, ignore_index = True)
    
    df = embaralhar(df)
    
    salvar_arquivo(df, nome)

## Hard Negative

In [28]:
#lista_nomes = ["celulares", "notebooks", "geladeiras", "fogoes", "tvs"]
lista_nomes = ["tvs"]

### 1:1

In [29]:
'''lista_df = []
for nome in lista_nomes:
    
    df_p = pd.read_csv(f"Dados/Pares/pares_{nome}.csv")
    df_np = pd.read_csv(f"Dados/Não Pares/Rankeado/1 por 1/nao_pares_{nome}.csv")
    
    lista_df.append(df_p)
    lista_df.append(df_np)

criar_dataset(lista_df, f"Por Categoria/Hard Negative/1 por 1/hn_1_1_{nome}")'''

### 5:1

In [31]:
'''lista_df = []
for nome in lista_nomes:
    
    df_p = pd.read_csv(f"Dados/Pares/pares_{nome}.csv")
    df_np = pd.read_csv(f"Dados/Não Pares/Rankeado/5 por 1/nao_pares_{nome}.csv")
    
    lista_df.append(df_p)
    lista_df.append(df_np)

criar_dataset(lista_df, f"Por Categoria/Hard Negative/5 por 1/hn_5_1_{nome}")'''

## Aleatório

### 1:1

In [33]:
'''lista_df = []
for nome in lista_nomes:
    
    df_p = pd.read_csv(f"Dados/Pares/pares_{nome}.csv")
    df_np = pd.read_csv(f"Dados/Não Pares/Aléatorio/1 por 1/nao_pares_{nome}.csv")
    
    lista_df.append(df_p)
    lista_df.append(df_np)

criar_dataset(lista_df, f"Por Categoria/Aleatório/1 por 1/ale_1_1_{nome}")'''

### 5:1

In [35]:
'''lista_df = []
for nome in lista_nomes:
    
    df_p = pd.read_csv(f"Dados/Pares/pares_{nome}.csv")
    df_np = pd.read_csv(f"Dados/Não Pares/Aléatorio/5 por 1/nao_pares_{nome}.csv")
    
    lista_df.append(df_p)
    lista_df.append(df_np)

criar_dataset(lista_df, f"Por Categoria/Aleatório/5 por 1/ale_5_1_{nome}")'''

# Separando o Dataset em Treino e Teste

70/30 por categoria

In [58]:
lista_nomes = ["celulares", "notebooks", "geladeiras", "fogoes", "tvs"]
#lista_nomes = ["tvs"]

In [67]:
def ler_csv(tipo, tipo_abr, proporcao, nome):

    df = pd.read_csv(f"{DIRETORIO}Por Categoria/{tipo}/{proporcao} por 1/{tipo_abr}_{proporcao}_1_{nome}.csv")

    return df


def ler_csv_teste_treino(tipo, tipo_abr, proporcao, nome, teste_treino):

    df = pd.read_csv(f"{DIRETORIO}Por Categoria/{tipo}/{proporcao} por 1/{teste_treino}/{tipo_abr}_{proporcao}_1_{nome}.csv")

    return df


def salvar_csv(df, tipo, tipo_abr, proporcao, nome, teste_treino):

    df.to_csv(f"{DIRETORIO}Por Categoria/{tipo}/{proporcao} por 1/{teste_treino}/{tipo_abr}_{proporcao}_1_{nome}.csv")


def separar_df_70_30(df, tipo, tipo_abr, proporcao, nome, qtd = 0.7, SEED = 42):

    df_treino = df.sample(frac = qtd, random_state = SEED)
    df_teste = df.drop(df_treino.index)

    print(f"Treino: {df_treino.shape[0]} linhas\n Teste: {df_teste.shape[0]} linhas")

    salvar_csv(df_treino, tipo, tipo_abr, proporcao, nome, "treino")
    salvar_csv(df_teste, tipo, tipo_abr, proporcao, nome, "teste")
    

In [63]:
'''for nome in lista_nomes:

    for tipo, tipo_abr in [['Aleatório', 'ale'], ['Hard Negative', 'hn']]:

        for proporcao in [1, 5]:

            df = ler_csv(tipo, tipo_abr, proporcao, nome)

            separar_df_70_30(df, tipo, tipo_abr, proporcao, nome)
            
            print("")
'''

'for nome in lista_nomes:\n\n    for tipo, tipo_abr in [[\'Aleatório\', \'ale\'], [\'Hard Negative\', \'hn\']]:\n\n        for proporcao in [1, 5]:\n\n            df = ler_csv(tipo, tipo_abr, proporcao, nome)\n\n            separar_df_70_30(df, tipo, tipo_abr, proporcao, nome)\n            \n            print("")\n'

# Juntandos os Datasets

In [77]:
def juntar_datasets(lista_nomes, treino_teste):

    for tipo, tipo_abr in [['Aleatório', 'ale'], ['Hard Negative', 'hn']]:

        for proporcao in [1, 5]:

            lista_df = []

            for nome in lista_nomes:

                df = ler_csv_teste_treino(tipo, tipo_abr, proporcao, nome, treino_teste)
                lista_df.append(df)

            criar_dataset(lista_df, f"{tipo}/{proporcao} por 1/{tipo_abr}_{proporcao}_1_{treino_teste}")

In [78]:
lista_nomes = ["celulares", "notebooks", "geladeiras", "fogoes", "tvs"]
#lista_nomes = ["tvs"]

juntar_datasets(lista_nomes = lista_nomes, treino_teste = "treino")
juntar_datasets(lista_nomes = lista_nomes, treino_teste = "teste")

# Lendo os Datasets

## Teste

In [79]:
df_ale_1 = pd.read_csv(f"{DIRETORIO}Aleatório/1 por 1/ale_1_1_teste.csv")
df_ale_5 = pd.read_csv(f"{DIRETORIO}Aleatório/5 por 1/ale_5_1_teste.csv")
df_hn_1 = pd.read_csv(f"{DIRETORIO}Hard Negative/1 por 1/hn_1_1_teste.csv")
df_hn_5 = pd.read_csv(f"{DIRETORIO}Hard Negative/5 por 1/hn_5_1_teste.csv")

In [81]:
print(f"Tamanho dos Datasets:\n\n\t| Hard\t| Ale\t|\n1 por 1\t| {df_hn_1.shape[0]}\t| {df_ale_1.shape[0]}\t|\n5 por 1\t| {df_hn_5.shape[0]}\t| {df_ale_5.shape[0]}\t|")

Tamanho dos Datasets:

	| Hard	| Ale	|
1 por 1	| 1329	| 1329	|
5 por 1	| 3987	| 3987	|


## Treino

In [82]:
df_ale_1 = pd.read_csv(f"{DIRETORIO}Aleatório/1 por 1/ale_1_1_treino.csv")
df_ale_5 = pd.read_csv(f"{DIRETORIO}Aleatório/5 por 1/ale_5_1_treino.csv")
df_hn_1 = pd.read_csv(f"{DIRETORIO}Hard Negative/1 por 1/hn_1_1_treino.csv")
df_hn_5 = pd.read_csv(f"{DIRETORIO}Hard Negative/5 por 1/hn_5_1_treino.csv")

In [83]:
print(f"Tamanho dos Datasets:\n\n\t| Hard\t| Ale\t|\n1 por 1\t| {df_hn_1.shape[0]}\t| {df_ale_1.shape[0]}\t|\n5 por 1\t| {df_hn_5.shape[0]}\t| {df_ale_5.shape[0]}\t|")

Tamanho dos Datasets:

	| Hard	| Ale	|
1 por 1	| 3101	| 3101	|
5 por 1	| 9303	| 9303	|
